# 「7.4.3 当てはまりの良さの指標の問題点」についての補足

p.193--194に次のようなコードがあります．この結果が書籍に掲載されているものと同じにならないという指摘をいただきました．このことについて，補足します．

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

my_data = sm.datasets.get_rdataset('cars', 'datasets').data

my_idx = [1, 10, 26, 33, 38, 43]
my_sample = my_data.iloc[my_idx, ]
X, y = my_sample[['speed']], my_sample['dist']

d = 5
X5 = PolynomialFeatures(d).fit_transform(X) # Xの「0」乗から5乗の変数

my_model = LinearRegression()
my_model.fit(X5, y)
y_ = my_model.predict(X5)

((y - y_)**2).mean()**0.5
#> 7.725744805546204e-07 # RMSE

## 簡単な解説

掲載しているのは，本書執筆時のGoogle Colabの結果です．環境によっては，これとは違う結果になることがあります．本書で強調しているように，回帰分析は乱数を使わないので，結果はいつも同じになるはずです．それにもかかわらず，違う結果が出るのはおかしい，というわけです．

ごもっともです．

違う結果が出る原因をひとことで言えば，「浮動小数点数の計算でよくある誤差」です．正確に計算すれば同じ結果になるはずのものでも，利用するコンピュータや，計算順序のちょっとした違いによって，違う結果になることがあるのです．

たとえば，上で作ったX5を左右反転しても結果は変わらないはずですが，数値計算の結果は（おそらく）変わります．（正確な計算の結果は後で示します．）

In [2]:
X5a = np.fliplr(X5) # 左右反転
my_model = LinearRegression()
my_model.fit(X5a, y)
y_ = my_model.predict(X5a)
((y - y_)**2).mean()**0.5

8.848157305870842e-11

以上で納得できる方にとっては，この話はこれで終わりでいいでしょう．

## 詳しい解説

計算は次のように行われます．

1. numpy（数値計算のためのモジュールによる前処理）
1. scipy（科学計算のためのモジュールによるLAPACKの呼び出し）
1. LAPACK（線形代数のためのライブラリによる最小二乗法の計算）
1. CPU（実際の計算）
1. numpy（後処理）

結果が異なるのは次のような場合です．

1. CPUが異なる場合：x86_64とaarch64（Apple Chip）では結果が異なります．aarch64上でx86_64をエミュレートするRosetta 2も，浮動小数点数のエミュレートは完璧ではないようです．このことは，x86_64とaarch64で同一のDockerイメージを使うことで確かめられます．
1. LAPACKが異なる場合：x86_64上でOpenBLAS（pipでnumpyを入れた場合）とIntel MKL（condaでnumpyを入れた場合）を比べることで確かめられます．
1. numpyのバージョンが異なる場合：numpy 1.19.5と1.20.0を比べることで確かめられます．


## お詫び

上のコードの`X5`には少し問題がありました．



In [3]:
pd.DataFrame(X5)

,0,1,2,3,4,5
0,1.0,4.0,16.0,64.0,256.0,1024.0
1,1.0,11.0,121.0,1331.0,14641.0,161051.0
2,1.0,16.0,256.0,4096.0,65536.0,1048576.0
3,1.0,18.0,324.0,5832.0,104976.0,1889568.0
4,1.0,20.0,400.0,8000.0,160000.0,3200000.0
5,1.0,22.0,484.0,10648.0,234256.0,5153632.0


左端のすべてが1の列は，線形モデルの定数に相当する部分です．`LinearRegression()`で改めて切片を導入するので，この列は不要です．

この列の有無の予測値への影響は，正確な計算なら無いのですが，数値計算では有ります．（正確な計算の結果は後で示します．）

ですから，次のようにすべきでした．

In [4]:
X5 = PolynomialFeatures(d, include_bias=False).fit_transform(X) # Xの1乗から5乗の変数
pd.DataFrame(X5)

,0,1,2,3,4
0,4.0,16.0,64.0,256.0,1024.0
1,11.0,121.0,1331.0,14641.0,161051.0
2,16.0,256.0,4096.0,65536.0,1048576.0
3,18.0,324.0,5832.0,104976.0,1889568.0
4,20.0,400.0,8000.0,160000.0,3200000.0
5,22.0,484.0,10648.0,234256.0,5153632.0


RMSE（訓練）はやはりほぼ0になります．

In [5]:
my_model = LinearRegression()
my_model.fit(X5, y)
y_ = my_model.predict(X5)
((y - y_)**2).mean()**0.5

4.943172285713167e-07

## scikit-learnがやっていること

`my_model.fit(X5, y)`と`my_model.predict(X5)`で行われるのは，次のような処理です（先と同じ結果になります）．

最小二乗法の計算をする`scipy.linalg.lstsq`の前後にnumpyを使う処理があります．そのため，scipyとLAPACKをそろえて最小二乗法の結果を同じにしても，numpyのバージョンが異なると結果が異なる場合があるのです．

In [6]:
import scipy
M = np.insert(X5, 5, 1, axis=1)
X_offset = M.mean(axis=0)
y_offset = y.mean()
b, _, _, _ = scipy.linalg.lstsq(M - X_offset, y - y_offset)
y_ = y_offset - np.dot(b, X_offset) + X5 @ b[:5]
((y - y_)**2).mean()**0.5

4.943172285713167e-07

Mはp.230の脚註4にあるような行列です．すべて1の列はどこにあってもいいのですが，`LinearRegression`と同じ結果にするためには，（おそらく）右端でなければなりません．

整形して表示します．

In [7]:
pd.DataFrame(M)

,0,1,2,3,4,5
0,4.0,16.0,64.0,256.0,1024.0,1.0
1,11.0,121.0,1331.0,14641.0,161051.0,1.0
2,16.0,256.0,4096.0,65536.0,1048576.0,1.0
3,18.0,324.0,5832.0,104976.0,1889568.0,1.0
4,20.0,400.0,8000.0,160000.0,3200000.0,1.0
5,22.0,484.0,10648.0,234256.0,5153632.0,1.0


## 正確な計算

正確に計算すると，RMSE（訓練）は0になります．Wolfram|Alphaを使うのが簡単です．

[5次式でフィット {4,10},{11,28},{16,32},{18,76},{20,32},{22,66}](https://www.wolframalpha.com/input/?i=5%E6%AC%A1%E5%BC%8F%E3%81%A7%E3%83%95%E3%82%A3%E3%83%83%E3%83%88+%7B4%2C10%7D%2C%7B11%2C28%7D%2C%7B16%2C32%7D%2C%7B18%2C76%7D%2C%7B20%2C32%7D%2C%7B22%2C66%7D&lang=ja)

計算方法はいろいろありますが，ここでは，ムーア・ペンローズ型一般逆行列（p.230の脚註4）を使います．

In [8]:
from sympy import *
Y = Matrix(y)
M = Matrix(np.array(X.speed)[:, np.newaxis]**[1, 2, 3, 4, 5, 0])
M

Matrix([
[ 4,  16,    64,    256,    1024, 1],
[11, 121,  1331,  14641,  161051, 1],
[16, 256,  4096,  65536, 1048576, 1],
[18, 324,  5832, 104976, 1889568, 1],
[20, 400,  8000, 160000, 3200000, 1],
[22, 484, 10648, 234256, 5153632, 1]])

In [9]:
b = M.pinv() * Y
b

Matrix([
[2786331301/582120],
[-126621007/155232],
[     919150/14553],
[  -1420541/620928],
[   293749/9313920],
[     -1407358/147]])

In [10]:
Y_ = M * b
(Y - Y_).norm()

0

この方法なら，「お詫び」で書いたように，すべて1の列が2列あっても，予測結果は変わりません．

In [11]:
M = Matrix(np.array(X.speed)[:, np.newaxis]**[0, 1, 2, 3, 4, 5, 0])
M

Matrix([
[1,  4,  16,    64,    256,    1024, 1],
[1, 11, 121,  1331,  14641,  161051, 1],
[1, 16, 256,  4096,  65536, 1048576, 1],
[1, 18, 324,  5832, 104976, 1889568, 1],
[1, 20, 400,  8000, 160000, 3200000, 1],
[1, 22, 484, 10648, 234256, 5153632, 1]])

In [12]:
b = M.pinv() * Y
Y_ = M * b
(Y - Y_).norm()

0

左右反転させても同じです．

In [13]:
M = Matrix(np.array(X.speed)[:, np.newaxis]**[0, 5, 4, 3, 2, 1, 0])
M

Matrix([
[1,    1024,    256,    64,  16,  4, 1],
[1,  161051,  14641,  1331, 121, 11, 1],
[1, 1048576,  65536,  4096, 256, 16, 1],
[1, 1889568, 104976,  5832, 324, 18, 1],
[1, 3200000, 160000,  8000, 400, 20, 1],
[1, 5153632, 234256, 10648, 484, 22, 1]])

In [14]:
b = M.pinv() * Y
Y_ = M * b
(Y - Y_).norm()

0

## 実行環境の確認

実行環境は次のように調べられます．掲載しているのは，2022年1月20日にGoogle Colabで実行した結果です．

In [15]:
import sys, numpy, scipy
print(sys.version, '\n\n')
print(numpy.__version__, scipy.__version__, '\n\n')
scipy.show_config()

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0] 


1.19.5 1.4.1 


lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
